In [4]:
import numpy as np 
import sys
import os
import matplotlib.pyplot as plt 
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"/openmd")
from SimulatorLJ import SimulatorLJ as Simulator


/home/julian/Documents/openmd/openmd


In [15]:

def force_lj_fast( positions, constants, box_length):
    """
    :param positions:
    :type positions:
    :param constants:
    :type constants:
    :param box_length:
    :type box_length:
    :return:
    :rtype:
    """
    
    epsilon, sigma = constants
    num = positions.shape[0]
    force = np.zeros((num, num, 3))
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            ###############
            difference = positions[j, :] - positions[i, :]
            distance = np.linalg.norm(difference)
            if distance > box_length / 2:
                distance -= box_length / 2
            elif distance <= -box_length / 2:
                distance += box_length / 2
            
            #################
            if distance == 0: 
                force[i,j] = 0 
                force[j,i] = 0
                break
            lj_part = (sigma / distance) ** 6
            lj_part_two = lj_part ** 2
            factor = 24 * epsilon
            factor_two = 2 * factor
            force[i, j, :] = (factor_two * lj_part_two - factor * lj_part) * (difference/distance)
            force[j, i, :] -= force[i, j, :]
    #print(np.sum(force, axis=1))
    return np.sum(force, axis=1)

In [16]:
dt = 0.001
positions = np.load("test_samples/test_sample__n_50_s_0.25_eps_0.25_.npy")
velocities = np.zeros(positions.shape)
constants = [1.0, 0.5]
box_length = 6
sim = Simulator(
    path = r"./output",
    title = "test_1",
    mass = 4,
    sim_time= 1000*dt ,
    time_step = dt,
    initial_values = [positions,velocities],
    box_length = box_length,
    force=None,
    force_constants=constants,
    integrator=None,   
    periodic=True,
)
sim.force = force_lj_fast
sim.force

<function __main__.force_lj_fast(positions, constants, box_length)>

In [72]:

def distance_new( positions, constants, box_length):
    """
    :param positions:
    :type positions:
    :param constants:
    :type constants:
    :param box_length:
    :type box_length:
    :return:
    :rtype:
    """
    
    epsilon, sigma = constants
    sigma_six = sigma**6
    prefactor_lj = 24*epsilon*sigma_six
    num = positions.shape[0]
    nvdim = positions.shape[1] # get nvdim for row major writing of matrices 
    position_rm = np.ravel(positions) #row major
    
    dmRM_CPU=np.zeros((num, num),dtype=np.float64,order='C')
    dmRM_CPU = np.ravel(dmRM_CPU)
    FRM_CPU = np.ravel(np.zeros((num, nvdim),dtype=np.float64,order='C'))
    
    
    for i in range(num):
        for j in range(i+1,num):
            dist = 0
            for k in range(nvdim):
                ab = position_rm[i*nvdim + k] - position_rm[j*nvdim + k]
                dist += ab*ab
            temp_dmRM = 1/np.sqrt(dist)
            temp_dmRM2 = temp_dmRM*temp_dmRM
            temp_dmRM6 = temp_dmRM2*temp_dmRM2*temp_dmRM2
            temp_dmRM8 = temp_dmRM6*temp_dmRM2
            for k in range(nvdim):
                ab = position_rm[i*nvdim + k] - position_rm[j*nvdim + k]
                print(i*num + k)
                FRM_CPU[i*num + k] = (prefactor_lj/temp_dmRM8)*(((2*sigma_six)/temp_dmRM6)-1) * ab
            
    return FRM_CPU

In [79]:
positions = np.random.random((2,3))

In [80]:
%timeit -n 30 distances_new = distance_new( positions, constants, box_length)

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1


In [40]:
distances_new, diffdis = distance_new( positions, constants, box_length)
print(diffdis)


[ 0.3944684   0.          0.         -0.01754702  0.          0.
  0.          0.          0.        ]


In [42]:
positions = np.random.random((3,3))

In [130]:
def distance_old( positions, constants, box_length):
    positions = positions.T
    epsilon, sigma = constants
    sigma_six = sigma**6
    prefactor_lj = 24*epsilon*sigma_six
    num = positions.shape[1]
    distances_sq = np.zeros((num, num))
    forces = np.zeros((positions.shape[0], num))
    for i in range(0, num - 1):
        for j in range(i + 1, num):
            ###############
            difference = positions[:, j] - positions[:, i]
            distance = np.linalg.norm(difference)
            distance_sq = distance*distance
            distance_six = distance_sq*distance_sq*distance_sq
            distance_eight = distance_six*distance_sq
            prefactor = prefactor_lj/distance_eight
            bracket = ((2*sigma)/distance_six - 1)
            result = prefactor*bracket
            force = result * difference
            forces[:, i] += force
            forces[:, j] -= force 
            #distances_sq[i, j] = inv_dist**2
            
            #distances[j, i] = distance
            
            #if distance > box_length / 2:
            #    distance -= box_length / 2
            #elif distance <= -box_length / 2:
            #    distance += box_length / 2
            
            #################

    return forces

In [131]:
constants = [1,1]
positions = np.array([[0,0,0], [1,1, 1]])
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

[[-0.27434842  0.27434842]
 [-0.27434842  0.27434842]
 [-0.27434842  0.27434842]]


In [132]:
constants = [1,1]
positions = np.array([[0,0,0], [1,0, 0]])
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

[[ 24. -24.]
 [  0.   0.]
 [  0.   0.]]


In [133]:
distance = np.sqrt(3)
prefactor = 24/distance**8
bracket = (2/distance**6 - 1)
result = prefactor*bracket
result 

-0.27434842249657077

In [128]:
constants = [1,1]
distances_old = distance_old( positions, constants, box_length)
print(distances_old)

[[ 24. -24.]
 [  0.   0.]
 [  0.   0.]]


In [134]:
positions = np.random.random((100,3))
%timeit -n 30 distances_old = distance_old( positions, constants, box_length)

42.9 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 30 loops each)


In [65]:
distances_new = distance_new( positions, constants, box_length)
distances_old = distance_old( positions, constants, box_length)
np.allclose(np.ravel(distances_old), distances_new) 

ValueError: operands could not be broadcast together with shapes (300,) (10000,) 

In [64]:
distances_new

array([0.        , 0.88217755, 0.34999277, 0.        , 0.        ,
       0.90319836, 0.        , 0.        , 0.        ])

In [65]:
distances_old**6*0.5**6

array([[0.        , 0.03315004, 8.50091403],
       [0.        , 0.        , 0.02878201],
       [0.        , 0.        , 0.        ]])